In [15]:
#import os
#import gradio as gr
import numpy as np
import torch
from models.sam import SamAutomaticMaskGenerator, SamPredictor, sam_model_registry
# from PIL import ImageDraw
# from mobile_sam.utils.tools import box_prompt, format_results, point_prompt
#from app.utils.tools_gradio import fast_process
# Most of our demo code is from [FastSAM Demo](https://huggingface.co/spaces/An-619/FastSAM). Huge thanks for AN-619.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
from cfg import parse_args
args =  parse_args()


In [3]:
from cfg import parse_args

# Load the pre-trained model
sam_checkpoint = "mobile_sam.pt"
# Define the model type: Tiny Vit
model_type = "vit_t"
# Load the model
mobile_sam = sam_model_registry[model_type](args , checkpoint=sam_checkpoint)
# Move the model to the device
mobile_sam = mobile_sam.to(device=device)
# Set the model to evaluation mode
mobile_sam.eval()

c:\KU\ATIA\finetuneSAM\models\sam\build_sam.py:157: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Sam(
  (image_encoder): TinyViT(
    (patch_embed): PatchEmbed(
      (seq): Sequential(
        (0): Conv2d_BN(
          (c): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): GELU(approximate='none')
        (2): Conv2d_BN(
          (c): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
    (layers): ModuleList(
      (0): ConvLayer(
        (blocks): ModuleList(
          (0-1): 2 x MBConv(
            (conv1): Conv2d_BN(
              (c): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): Conv2d_BN(
 

In [10]:
next( mobile_sam.mask_decoder.parameters())

Parameter containing:
tensor([[-0.0879, -0.1321,  0.0836,  ...,  0.0004, -0.0185,  0.0011],
        [-0.0375, -0.1283,  0.0036,  ...,  0.0273, -0.0380,  0.0483],
        [-0.0320,  0.0465,  0.0535,  ..., -0.0090, -0.0755,  0.1173],
        ...,
        [ 0.0465,  0.0303, -0.0648,  ...,  0.0008,  0.0163, -0.0158],
        [ 0.0622,  0.0186, -0.0068,  ...,  0.0297,  0.0375, -0.0383],
        [ 0.0084, -0.0070,  0.0405,  ..., -0.0055,  0.0015,  0.0230]],
       device='cuda:0', requires_grad=True)

In [4]:
#from segment_anything import SamPredictor, sam_model_registry
from models.sam import SamPredictor, sam_model_registry
from models.sam.utils.transforms import ResizeLongestSide
from skimage.measure import label
from models.sam_LoRa import LoRA_Sam
#Scientific computing 
import numpy as np
import os
#Pytorch packages
import torch

from torch import nn
import torch.optim as optim
import torchvision
from torchvision import datasets
#from tensorboardX import SummaryWriter
#Visulization
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

#Others
from torch.utils.data import DataLoader, Subset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import copy

In [4]:


from utils.dataset import Public_dataset
import torch.nn.functional as F
from torch.nn.functional import one_hot
from pathlib import Path
from tqdm import tqdm
from utils.losses import DiceLoss
from utils.dsc import dice_coeff_multi_class
import cv2
import monai
#from utils.utils import vis_image
import cfg
import json

In [5]:
args.arch = "vit_t"
args.sam_ckpt = "mobile_sam_weights.pt"
args.finetune_type = "vanilla"
sam = sam_model_registry[args.arch](args,checkpoint=sam_checkpoint,num_classes=args.num_cls)


# args.arch = "vit_b"
# args.sam_ckpt = "sam_vit_b_01ec64.pth" 
# sam = sam_model_registry[args.arch](args,checkpoint=os.path.join(args.sam_ckpt),num_classes=args.num_cls)

sam.eval()
sam.to(device)

c:\KU\ATIA\finetuneSAM\models\sam\build_sam.py:157: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Sam(
  (image_encoder): TinyViT(
    (patch_embed): PatchEmbed(
      (seq): Sequential(
        (0): Conv2d_BN(
          (c): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): GELU(approximate='none')
        (2): Conv2d_BN(
          (c): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
    (layers): ModuleList(
      (0): ConvLayer(
        (blocks): ModuleList(
          (0-1): 2 x MBConv(
            (conv1): Conv2d_BN(
              (c): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): Conv2d_BN(
 

In [6]:
sam = mobile_sam
from models.sam import predictor
predictor = predictor.SamPredictor(sam)


In [7]:
def put_mark(img, center, size):
    img = img.copy()
    x, y = center
    x = int(x)
    y = int(y)
    size = int(size)
    img[y-size:y+size, x-size:x+size] = [0, 255, 0]
    
    return img


In [8]:
from PIL import Image
image = Image.open('picture4.jpg')
image = np.asarray(image)
image = image[0:1024,0:1024]
image = image.copy()
image = put_mark(image, (940,730), 5)
print(image.shape)
Image.fromarray(image).show()

(1024, 1024, 3)


In [9]:
def format_results(masks, scores, logits, filter=0):
    annotations = []
    n = len(scores)
    for i in range(n):
        annotation = {}

        mask = masks[i]
        tmp = np.where(mask != 0)
        if np.sum(mask) < filter:
            continue
        annotation["id"] = i
        annotation["segmentation"] = mask
        annotation["bbox"] = [
            np.min(tmp[0]),
            np.min(tmp[1]),
            np.max(tmp[1]),
            np.max(tmp[0]),
        ]
        annotation["score"] = scores[i]
        annotation["area"] = annotation["segmentation"].sum()
        annotations.append(annotation)
    return annotations

In [10]:
def put_massk(image,mask):
    img = image.copy()
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i,j] == 1:
                img[i,j] = [255,0,0]
    return img


In [14]:

p = np.array([940,730]).reshape(1,2)
p = None

predictor.set_image(image)
masks, scores, logits = predictor.predict(p,point_labels= np.array([1]))
results = format_results(masks, scores, logits, 0)

fst = results[0]["segmentation"]
#fst,snd= results[0]["segmentation"],results[1]["segmentation"]#,results[2]["segmentation"]


c:\KU\ATIA\finetuneSAM\models\sam\predictor.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_image = torch.tensor(transformed_image*1.0/255)


In [13]:
scores

array([ 0.02837351, -0.09287645], dtype=float32)

In [21]:
Image.fromarray(put_massk(image,fst)).show()
Image.fromarray(put_massk(image,snd)).show()

NameError: name 'snd' is not defined

In [36]:
#def train_model(trainloader,valloader,dir_checkpoint,epochs):
# if args.if_warmup:
#     b_lr = args.lr / args.warmup_period
# else:
#     b_lr = args.lr

#sam = sam_model_registry[args.arch](args,checkpoint=os.path.join(args.sam_ckpt),num_classes=args.num_cls)
# if args.finetune_type == 'adapter':
#     for n, value in sam.named_parameters():
#         if "Adapter" not in n: # only update parameters in adapter
#             value.requires_grad = False
#     print('if update encoder:',args.if_update_encoder)
#     print('if image encoder adapter:',args.if_encoder_adapter)
#     print('if mask decoder adapter:',args.if_mask_decoder_adapter)
#     if args.if_encoder_adapter:
#         print('added adapter layers:',args.encoder_adapter_depths)
    
# elif args.finetune_type == 'vanilla' and args.if_update_encoder==False:   
#     print('if update encoder:',args.if_update_encoder)
#     for n, value in sam.image_encoder.named_parameters():
#         value.requires_grad = False
# elif args.finetune_type == 'lora':
#     print('if update encoder:',args.if_update_encoder)
#     print('if image encoder lora:',args.if_encoder_lora_layer)
#     print('if mask decoder lora:',args.if_decoder_lora_layer)
#     sam = LoRA_Sam(args,sam,r=4).sam
# sam.to('cuda')
    
optimizer = optim.AdamW(sam.parameters(), lr=b_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1, amsgrad=False)
optimizer.zero_grad()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) #learning rate decay
criterion1 = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, to_onehot_y=True,reduction='mean')
criterion2 = nn.CrossEntropyLoss()

iter_num = 0
max_iterations = epochs * len(trainloader) 
writer = SummaryWriter(dir_checkpoint + '/log')

pbar = tqdm(range(epochs))
val_largest_dsc = 0
last_update_epoch = 0
for epoch in pbar:
    sam.train()
    train_loss = 0
    for i,data in enumerate(tqdm(trainloader)):
        imgs = data['image'].cuda()
        msks = torchvision.transforms.Resize((args.out_size,args.out_size))(data['mask'])
        msks = msks.cuda()

        if args.if_update_encoder:
            img_emb = sam.image_encoder(imgs)
        else:
            with torch.no_grad():
                img_emb = sam.image_encoder(imgs)
        
        # get default embeddings
        sparse_emb, dense_emb = sam.prompt_encoder(
            points=None,
            boxes=None,
            masks=None,
        )
        pred, _ = sam.mask_decoder(
                        image_embeddings=img_emb,
                        image_pe=sam.prompt_encoder.get_dense_pe(), 
                        sparse_prompt_embeddings=sparse_emb,
                        dense_prompt_embeddings=dense_emb, 
                        multimask_output=True,
                        )
        loss_dice = criterion1(pred,msks.float()) 
        loss_ce = criterion2(pred,torch.squeeze(msks.long(),1))
        loss =  loss_dice + loss_ce
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        
        if args.if_warmup and iter_num < args.warmup_period:
            lr_ = args.lr * ((iter_num + 1) / args.warmup_period)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_

        else:
            if args.if_warmup:
                shift_iter = iter_num - args.warmup_period
                assert shift_iter >= 0, f'Shift iter is {shift_iter}, smaller than zero'
                lr_ = args.lr * (1.0 - shift_iter / max_iterations) ** 0.9  # learning rate adjustment depends on the max iterations
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_
            else:
                lr_ = args.lr

        train_loss += loss.item()
        iter_num+=1
        writer.add_scalar('info/lr', lr_, iter_num)
        writer.add_scalar('info/total_loss', loss, iter_num)
        writer.add_scalar('info/loss_ce', loss_ce, iter_num)
        writer.add_scalar('info/loss_dice', loss_dice, iter_num)

    train_loss /= (i+1)
    pbar.set_description('Epoch num {}| train loss {} \n'.format(epoch,train_loss))

NameError: name 'optim' is not defined

torch.Size([3, 1024, 1024])

ValueError: expected 4D input (got 3D input)